In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Chest-Cancer-Classification-MLFlow-DVC\\research'

In [3]:
# We need to be on the project root folder to ensure our code works with relative paths and avoid the FileNotFoundError.
os.chdir("../")

In [4]:
%pwd

'd:\\Projects\\Chest-Cancer-Classification-MLFlow-DVC'

In [5]:
from dataclasses import dataclass
from pathlib import Path

# @dataclass(frozen=True): This decorator is used to automatically generate special methods like __init__(), __repr__(), __eq__(), etc., for the class. 
# The frozen=True parameter makes the instances of this class immutable, meaning that its fields cannot be modified after they are created.
# to ensure that the configuration settings remain constant throughout the execution of the program.
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    """
    ConfigurationManager handles the setup and retrieval of configuration settings for the pipeline.
    """

    def __init__(self, 
                 config_filepath=CONFIG_FILE_PATH, 
                 params_filepath=PARAMS_FILE_PATH):
        # Load configuration settings from the specified config.yaml file
        self.config = read_yaml(config_filepath)
        
        # Load parameters from the specified params file
        self.params = read_yaml(params_filepath)

        # Create the root directory for artifacts as declared in config.yaml file
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """
        Retrieves and prepares the data ingestion configuration settings.

        Returns:
            DataIngestionConfig: Data ingestion configuration settings.
        """
        # Extract data_ingestion settings from config.yaml file
        config_di = self.config.data_ingestion  

        # Create directory for data ingestion artifacts
        create_directories([config_di.root_dir])

        # Create a DataIngestionConfig object with the settings
        data_ingestion_config = DataIngestionConfig(
            root_dir=config_di.root_dir,
            source_URL=config_di.source_URL,
            local_data_file=config_di.local_data_file,
            unzip_dir=config_di.unzip_dir
        )

        return data_ingestion_config


In [8]:
import os
import zipfile
# import gdown package to download from google drive
import gdown
from src.cnnClassifier.logging import logger
from src.cnnClassifier.utils.common import get_size


In [9]:
class DataIngestion:
    """
    DataIngestion handles downloading and extracting the dataset for the machine learning pipeline.

    Attributes:
        config (DataIngestionConfig): Configuration for data ingestion process.
    """
    
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """
        Downloads the dataset from the specified URL.

        Returns:  str: Path to the downloaded file.
        """
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file

            # Ensure the directory for downloading the dataset exists
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            # Construct the download URL for Google Drive
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            
            # Download the file using gdown
            gdown.download(prefix + file_id, zip_download_dir, quiet=False)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

            return str(zip_download_dir)

        except Exception as e:
            logger.error(f"Error downloading data: {e}")
            raise e
  
    def extract_zip_file(self):
        """
        Extracts the downloaded zip file into the specified directory.

        Returns: None
        """
        unzip_path = self.config.unzip_dir

        # Ensure the directory for unzipping the dataset exists
        os.makedirs(unzip_path, exist_ok=True)
        logger.info(f"Extracting zip file {self.config.local_data_file} into directory {unzip_path}")

        # Extract the contents of the zip file
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

        logger.info(f"Extracted zip file {self.config.local_data_file} into directory {unzip_path}")

In [10]:
try:
    # Initialize the ConfigurationManager to load configuration settings
    config = ConfigurationManager()
    
    # Retrieve the data ingestion configuration settings
    data_ingestion_config = config.get_data_ingestion_config()
    
    # Initialize the DataIngestion process with the configuration settings
    data_ingestion = DataIngestion(config=data_ingestion_config)
    
    # Download the dataset file from the specified source URL
    data_ingestion.download_file()
    
    # Extract the downloaded zip file to the specified directory
    data_ingestion.extract_zip_file()
    
except Exception as e:
    # Log the exception details and raise it
    logger.error(f"An error occurred: {e}")
    raise e

[2024-07-31 15:17:31,254: INFO: common: YAML file: config\config.yaml loaded successfully]
[2024-07-31 15:17:31,308: INFO: common: YAML file: params.yaml loaded successfully]
[2024-07-31 15:17:31,313: INFO: common: Created directory at: artifacts]
[2024-07-31 15:17:31,317: INFO: common: Created directory at: artifacts/data_ingestion]
[2024-07-31 15:17:31,320: INFO: 3843480513: Downloading data from https://drive.google.com/file/d/1J5yEH4PMqHKBt2d4rVCK_tc1NyP8hREe/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1J5yEH4PMqHKBt2d4rVCK_tc1NyP8hREe
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1J5yEH4PMqHKBt2d4rVCK_tc1NyP8hREe&confirm=t&uuid=d1279325-94f1-44de-b471-fb21dd59c8ed
To: d:\Projects\Chest-Cancer-Classification-MLFlow-DVC\artifacts\data_ingestion\data.zip
100%|██████████| 107M/107M [00:09<00:00, 11.0MB/s] 

[2024-07-31 15:17:44,878: INFO: 3843480513: Downloaded data from https://drive.google.com/file/d/1J5yEH4PMqHKBt2d4rVCK_tc1NyP8hREe/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-07-31 15:17:44,883: INFO: 3843480513: Extracting zip file artifacts/data_ingestion/data.zip into directory artifacts/data_ingestion]


[2024-07-31 15:17:47,538: INFO: 3843480513: Extracted zip file artifacts/data_ingestion/data.zip into directory artifacts/data_ingestion]
